In [18]:
#! /usr/local/Cellar/jupyterlab/2.2.8/libexec/bin/python3.8 -m pip install networkx
import sklearn.metrics
import networkx as nx
from glob import glob
import igraph as ig
import numpy as np
import pandas as pd
import re
import os

In [10]:
def read_graph(path):
    edges = []
    labels = []
    with open(path, 'r') as f:
        next(f)
        next_line = next(f)
        if next_line.startswith('*'):
            for line in f:
                i, j, *_ = re.split('\s+', line.strip())
                i, j = int(i), int(j)
                edge = (i, j)
                edges.append(edge)
        else:
            labels.append(re.split('\s+', next_line.strip())[1])
            for line in f:
                if line.startswith('*'):
                    break
                labels.append(re.split('\s+', line.strip())[1])
            for line in f:
                i, j, *_ = re.split('\s+', line.strip())
                i, j = int(i), int(j)
                edge = (i, j)
                edges.append(edge)
    if labels:
        vertex_attrs = {'label': labels}
    else:
        vertex_attrs = {}
    g = ig.Graph(edges=edges, directed=False, vertex_attrs=vertex_attrs)
    g.vs.select(_degree=0).delete()
    g = g.simplify()
    return g

In [53]:
# Make dirs for plots
for algorithm in ['Leiden', 'Multi-level', 'Fast-greedy']:
    if not os.path.exists(algorithm):
        os.makedirs(algorithm)

In [88]:
#ig.plot(communities, mark_groups=True, layout=layout)
! pwd
networks = []
leiden_mod = []
multilevel_mod = []
fastgreedy_mod = [] 


for path in glob("A3-networks/real/*.net"):
    name = os.path.basename(path)
    networks.append(name) 
    g = read_graph(path)

    #LEIDEN
    leiden = g.community_leiden('modularity', n_iterations=-1)
    leiden_mod.append(leiden.modularity)
    
    leiden_plot = ig.plot(leiden, layout = g.layout("circle"), bbox = (300, 300), margin = 20)
    leiden_plot.save('Leiden/' + name.replace('.', '_') + ".png")
    
    #MULTI-LEVEL
    multilevel = g.community_multilevel()
    multilevel_mod.append(multilevel.modularity)
    
    ml_plot = ig.plot(multilevel, layout = g.layout("circle"), bbox = (300, 300), margin = 20)
    ml_plot.save('Multi-level/' + name.replace('.', '_') + ".png")
    
    #FAST GREEDY
    fastgreedy = g.community_fastgreedy().as_clustering()
    fastgreedy_mod.append(fastgreedy.modularity)
    fg = g.community_fastgreedy()
   
    #if ground truth exists - compare
    if os.path.exists("A3-networks/real/"+ name.replace(".net",".clu") ):
        print("time to compare! ","A3-networks/real/"+ name.replace(".net",".clu"))
        reference_g = read_graph("A3-networks/real/" + name.replace(".net",".clu"))
        #reference_leiden = reference_g.community_leiden('modularity', n_iterations=-1)
        #print(compare(leiden, reference_g, method = c("vi", "nmi","split.join", "rand","adjusted.rand")))
        
        
    #PLOT AND SAVE
    #fg_plot = ig.plot(fastgreedy, layout = g.layout("circle"), bbox = (300, 300), margin = 20)
    #fg_plot.save('Fast-greedy/' + name.replace('.', '_') + ".png")


#SAVE TO TABLE
mod = {'Leiden': leiden_mod,
        'Multi-level': multilevel_mod,
        'Fast-greedy': fastgreedy_mod
        }
df = pd.DataFrame(mod, columns = ['Leiden','Multi-level', 'Fast-greedy'], index=networks)
print("\nMODULARITY TABLE\n", df)


/Users/betty/Desktop/CN/Complex-Networks/A3
time to compare!  A3-networks/real/cat_cortex_sim.clu


IndexError: list index out of range

In [8]:
g = ig.load('../real/zachary.net')
layout = g.layout_auto()